In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import norm
from statsmodels.base.model import GenericLikelihoodModel
import seaborn as sns
sns.set_palette("muted")
sns.set_color_codes()
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
sns.set_style({"axes.grid": "True", "grid.color": "0.95"})

In [ ]:
plt.rcParams["figure.figsize"] = [6,6]
plt.rcParams["figure.dpi"] = 100

In [ ]:
blue = (114/256, 147/256, 203/256)
orange = (225/256, 151/256,  76/256)
green = (132/256, 186/256,  91/256)
red = (211/256,  94/256,  96/256)
grey = (128/256, 133/256, 133/256)
violet = (144/256, 103/256, 167/256)
brown = (171/256, 104/256,  87/256)
yellow = (204/256, 194/256,  16/256)

In [ ]:
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
import root_pandas as rpd
import glob

variables = [
    'phit',
    'thetab',
    'thetat',
    'vrusable',
    'vtusable',
    'vrchi2',
    'vtchi2',
    'vrndf',
    'vtndf',
    'vrntrk',
    'vtntrk',
    'vrerr6',
    'vterr6',
    'evmcflag',
    'csbdtg',
    'de',
    'dt'
]

df_mcbkg = rpd.read_root([
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd1_on_resonance_evtgen-mixed_s10.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd1_on_resonance_evtgen-charged_s10.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd1_on_resonance_evtgen-charm_s10.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd1_on_resonance_evtgen-uds_s10.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd2_on_resonance_evtgen-mixed_s0.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd2_on_resonance_evtgen-charged_s0.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd2_on_resonance_evtgen-charm_s0.root",
        "../data/Kpi/mc_wo_signal/DSRhoSkim_svd2_on_resonance_evtgen-uds_s0.root"
    ], columns=variables)

df_mcsideb = rpd.read_root([
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd1_on_resonance_evtgen-mixed_s10.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd1_on_resonance_evtgen-charged_s10.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd1_on_resonance_evtgen-charm_s10.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd1_on_resonance_evtgen-uds_s10.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd2_on_resonance_evtgen-mixed_s0.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd2_on_resonance_evtgen-charged_s0.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd2_on_resonance_evtgen-charm_s0.root",
        "../data/Kpi/sidebands/mc/DSRhoSkim_svd2_on_resonance_evtgen-uds_s0.root"
    ], columns=variables)

df_datasideb = rpd.read_root(glob.glob("../data/Kpi/sidebands/*.root"), columns=variables)

### Apply cuts to datasets

In [ ]:
def apply_cuts(df):
    return df[ 
        (df.vrusable == 1) &
        (df.vtusable == 1) &
        (((df.vrchi2/df.vrndf) < 50) | (df.vrntrk == 1)) &
        (((df.vtchi2/df.vtndf) < 50) | (df.vtntrk == 1)) &
        (((np.sqrt(df.vrerr6) < 0.02) & (df.vrntrk > 1)) | ((np.sqrt(df.vrerr6) < 0.05) & (df.vrntrk == 1))) &
        (((np.sqrt(df.vterr6) < 0.02) & (df.vtntrk > 1)) | ((np.sqrt(df.vterr6) < 0.05) & (df.vtntrk == 1))) &
        (df.csbdtg > -0.6) &
        ((df.de > -0.14) & (df.de < 0.068)) &
        ((df.dt > -10) & (df.dt < 10)) &
        (df.thetab > 0.5)
    ]

df_mcbkg = apply_cuts(df_mcbkg)
df_mcsideb = apply_cuts(df_mcsideb)
df_datasideb = apply_cuts(df_datasideb)

In [ ]:
def plot_with_ratio(datasets, labels, xlabel, colors):
    fig, (ax1, ax2) = plt.subplots(nrows=2, 
                                   sharex=True, 
                                   gridspec_kw = {'height_ratios':[3, 1]})

    fig.subplots_adjust(hspace=0.1)

    ns, bins, patches = ax1.hist([datasets[0], datasets[1]],
                                histtype="step",
                                bins=50,
                                linewidth=1,
                                color=colors,
                                label=labels)

    ax1.legend()
    plt.xlabel(xlabel)

    ratios = ns[0]/ns[1]
    errors = ns[0]/ns[1]*np.sqrt(1/ns[0] + 1/ns[1])
    
    ax2.errorbar(x=bins[:-1], 
                 y=ratios, 
                 yerr=errors, 
                 fmt='o',
                 color=grey)
    
    # Sets maximal number of ticks
    ax2.yaxis.set_major_locator(plt.MaxNLocator(5))

    ax1.set_ylabel('Data')
    ax2.set_ylabel('Ratio')
    
    fig.savefig('{xlabel}_{label1}_{label2}.pdf'.format(xlabel=datasets[0].name, 
                                                        label1=labels[0].replace(" ", "_").replace(".",""), 
                                                        label2=labels[1].replace(" ", "_").replace(".","")),
                format="pdf", bbox_inches = 'tight')

In [ ]:
datasets = (df_mcsideb.thetat, df_datasideb.thetat)
xlabel = r'$\theta_t$ [rad]'
labels = ['MC sideb.', 'data sideb.']
colors = [blue, green]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcsideb.thetab, df_datasideb.thetab)
xlabel = r'$\theta_b$ [rad]'
labels = ['MC sideb.', 'data sideb.']
colors = [blue, green]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcsideb.phit, df_datasideb.phit)
xlabel = r'$\phi_t$ [rad]'
labels = ['MC sideb.', 'data sideb.']
colors = [blue, green]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcbkg.thetat, df_datasideb.thetat)
xlabel = r'$\theta_t$ [rad]'
labels = ['MC bkg.', 'data sideb.']
colors = [red, green]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcbkg.thetab, df_datasideb.thetab)
xlabel = r'$\theta_b$ [rad]'
labels = ['MC bkg.', 'data sideb.']
colors = [red, green]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcbkg.phit, df_datasideb.phit)
xlabel = r'$\phi_t$ [rad]'
labels = ['MC bkg.', 'data sideb.']
colors = [red, green]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcbkg.thetat, df_mcsideb.thetat)
xlabel = r'$\theta_t$ [rad]'
labels = ['MC bkg.', 'MC sideb.']
colors = [red, blue]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcbkg.thetab, df_mcsideb.thetab)
xlabel = r'$\theta_b$ [rad]'
labels = ['MC bkg.', 'MC sideb.']
colors = [red, blue]
plot_with_ratio(datasets, labels, xlabel, colors)

In [ ]:
datasets = (df_mcbkg.phit, df_mcsideb.phit)
xlabel = r'$\phi_t$ [rad]'
labels = ['MC bkg.', 'MC sideb.']
colors = [red, blue]
plot_with_ratio(datasets, labels, xlabel, colors)